# Annotate Homographies
Large scale homography annotation for extraction of next best views.

In [1]:
import os
import cv2
import torchvision.io as io
from torchvision.datasets.video_utils import VideoClips
from torch.utils.data import DataLoader
import sys

sys.path.append('../')

from utils import load_yaml, dict_list_to_compose

servers = load_yaml('../configs/servers.yml')
server = servers['local']

databases = load_yaml('../configs/cholec80_transforms.yml')
database = databases['databases'][0]

In [2]:
paths = [
    os.path.join(
        server['database']['location'], 
        database['prefix'], 
        database['videos']['prefix'], 
        x
    ) for x in database['videos']['files'][:2]
]


# video reader not compiled yet https://github.com/pytorch/vision/issues/1446
# video reading https://github.com/pytorch/vision/blob/ed5b2dc3a5e7411d8b40cc7e526e151983e99cf9/torchvision/datasets/video_utils.py#L45-L69
# dataset example check https://github.com/pytorch/vision/blob/ed5b2dc3a5e7411d8b40cc7e526e151983e99cf9/torchvision/datasets/kinetics.py#L50-L78

ds = VideoClips(paths, clip_length_in_frames=10, frames_between_clips=1)
# dl = DataLoader(ds)

print(ds.metadata)
clip = ds.get_clip(0)

# for idx, file in enumerate(database['videos']['files']):
#     path = os.path.join(server['database']['location'], database['prefix'], database['videos']['prefix'], file)



#     #  pts, video_fps = io.read_video_timestamps(path)
#     # print(pts, video_fps) # 3600
#     pts = 3600
#     N_start = 1500001
#     N_end = 1500002  # predictive horizon
#     vframe, aframe, info = io.read_video(path, N_start*pts, (N_end-1)*pts, pts_unit='str')

#     print(info)

#     transforms = database['transforms'][idx]
#     transforms = dict_list_to_compose(transforms)

#     print(vframe.shape)

#     for idx, frame in enumerate(vframe):
#         frame = frame.numpy()
#         frame = transforms(frame)
#         cv2.imshow('frame', frame)
#         cv2.waitKey()

# cv2.destroyAllWindows()


100%|██████████| 1/1 [01:30<00:00, 90.29s/it]{'video_paths': ['/media/martin/Samsung_T5/data/endoscopic_data/cholec80/videos/video01.mp4'], 'video_pts': [tensor([        0,      3600,      7200,  ..., 155962800, 155966400,
        155970000])], 'video_fps': [25.0]}

